In [10]:
%reload_ext autoreload
%autoreload 2

from copy import deepcopy
from datetime import datetime, timedelta
from file_exchange import config, network
from file_exchange.network import Network
from file_exchange.peers_view import PeersView
from file_exchange.user import User
from gevent.event import AsyncResult
from gevent.pool import Group
from gevent.select import select
import gevent
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import multiprocessing
import numpy as np
import os
import pandas as pd
import pickle
import random
import re
import seaborn as sns
import shutil
import sys
import time
import traceback

def load_conf(path):
    f = open(path+'conf.json', 'r')
    conf = json.load(f)
    conf['period'] = datetime.strptime(conf['period'], "%H:%M:%S") - \
                      datetime.strptime("00:00", "%H:%M")
    return conf 

def fetch_dataframes(conf):
    devices_df = pd.DataFrame()
    users_df = pd.DataFrame()
    for fn in os.listdir(conf['output_dir']):
        if not fn.endswith(".csv"):
            continue
            
        filepath = conf['output_dir']+fn
        if re.search("device", fn):
            devices_df = append_to_df(filepath, devices_df)
        elif re.search("user", fn):
            users_df = append_to_df(filepath, users_df)

    devices_df['t'] -= min(devices_df['t'])
    devices_df = devices_df.sort_values('t')
    users_df['t'] -= min(users_df['t'])
    users_df = users_df.sort_values('t')
    return devices_df, users_df

def fetch_experiment_data(path):
    conf = load_conf(path)
    with open(conf['output_dir']+"files_info.bin", 'rb') as f:
        files_info = pickle.load(f)
    devices_df, users_df = fetch_dataframes(conf)  
    return conf, files_info, devices_df, users_df

def append_to_df(filepath, df):
    try:
        df2 = pd.read_csv(filepath)
    except:
        print("Failed loading {}: {}".format(fn, sys.exc_info()))
        pass
    else:
        if(df.empty):
            df = df2
        else:
            df = df.append(df2,)
    return df

# if experiment_set:
#     data_dir = 'data/'+experiment_set_name+''/'+conf['experiment_name']+'/'
# else:
#     data_dir = "data/"+conf['experiment_name']+'/'
# #data_dir="data/experiment_set/experiment_1/"
# conf, files_info, devices_df, users_df = fetch_experiment_data(data_dir)

In [12]:
def fetch_experiment_set(path):
    devices_df = pd.DataFrame()
    users_df = pd.DataFrame()
    conf = {}
    files_info = {}
    for experiment_name in os.listdir(path):
        conf_, files_info_, devices_df_, users_df_ = \
            fetch_experiment_data(path+experiment_name+"/")
        
        devices_df_['experiment_name'] = experiment_name
        users_df_['experiment_name'] = experiment_name
        devices_df = devices_df.append(devices_df_, ignore_index=True)
        users_df = users_df.append(users_df_, ignore_index=True)
        conf[experiment_name] = conf_
        files_info[experiment_name] = files_info_
    return conf, files_info, devices_df, users_df

# Mesage IDs are not unique in the dataset! (experiment_id, mes_id) should be
conf, files_info, devices_df, users_df =  fetch_experiment_set("data/users_number/")

In [33]:
a=np.unique(devices_df[['experiment_id', 'mess_id']].values.astype(np.uint), axis=0)
print(len(a))
a

302066


array([[    0,     1],
       [    0,    17],
       [    0,    23],
       ...,
       [   99, 65377],
       [   99, 65397],
       [   99, 65444]], dtype=uint64)

In [41]:
devices_df[(devices_df['experiment_id'] == 0) &
                             (devices_df['mess_id'] == 1)]

,t,experiment_id,addr,owner,current_round,file_id,chunk_id,mess_id,type,sent,received,forwarded,success,messages_in_queue,experiment_name
1609603,137.463851,0,ijwpsp7gd4w3,Many,8,osusd0,6,1,1,612,0,0,True,0,experiment0_5users
1610728,147.586667,0,94r5lq6nps1s,Sherie,9,osusd0,6,1,1,0,0,612,True,1,experiment0_5users
1612749,166.516649,0,ijwpsp7gd4w3,Many,10,osusd0,6,1,1,0,0,612,True,10,experiment0_5users
1614751,185.275824,0,jzpyhqjibs8n,Beatriz,11,osusd0,6,1,1,0,612,0,True,8,experiment0_5users
1614752,185.284414,0,jzpyhqjibs8n,Beatriz,11,osusd0,6,1,2,100,0,0,True,8,experiment0_5users
1617390,210.689928,0,jzpyhqjibs8n,Beatriz,13,osusd0,6,1,2,0,0,100,True,13,experiment0_5users
1618796,223.584633,0,uc3rforv2zc6,Moira,14,osusd0,6,1,2,0,0,100,True,17,experiment0_5users
1619888,233.139916,0,qxyn79jbi6j4,Many,14,osusd0,6,1,2,0,100,0,True,18,experiment0_5users


## RTT

In [56]:
messages_id = np.unique(devices_df[['experiment_id', 'mess_id']].values.astype(np.uint), axis=0)

In [63]:
pd.DataFrame([[1,2], [3,4]], columns=['a', 'b'])

,a,b
0,1,2
1,3,4


In [68]:
chunk.min()

137.46385116194142

In [64]:
transfer_times.append(pd.DataFrame([
                [chunk, 1],
                [ack, 2],
                [rtt, 3]
            ], columns=['t', 'type']), ignore_index=True)

,t,type
0,1609603 137.463851 1610728 147.586667 16...,1
1,1614752 185.284414 1617390 210.689928 16...,2
2,1609603 NaN 1610728 NaN 1612749 NaN 1614...,3


In [ ]:
#messages_id = np.unique(devices_df[['experiment_id', 'mess_id']].values.astype(np.uint), axis=0)
n_messages = messages_id.shape[0]
transfer_times = pd.DataFrame()

i = 0
for experiment_id in np.unique(messages_id[:, 0]):
    experiment_data = devices_df[
        devices_df['experiment_id'] == experiment_id
    ][['type', 't', 'mess_id']]
    for m_id in np.unique(messages_id[messages_id[:,0] == experiment_id][:, 1]):
        data = experiment_data[experiment_data['mess_id'] == m_id][['type', 't']]
    
        if not data.empty:
            chunk = data[data['type'] == 1]['t'].min()
            ack = data[data['type'] == 2]['t'].max()

            rtt = None
            if chunk is not None and ack is not None:
                rtt = chunk + ack

            transfer_times = transfer_times.append(pd.DataFrame([
                [chunk, 1],
                [ack, 2],
                [rtt, 3]
            ], columns=['t', 'type']), ignore_index=True)
        else:
            print("No ({}, {}) in devices_df".format(experiment_id, m_id))
        
        if i % 10000 == 0:
            print("{:.1%} done".format((i+1)/n_messages))
        i += 1
        
# for i, (experiment_id, m_id) in enumerate(messages_id):
#     if i % 10000 == 0:
#         print("{:.1%} done".format((i+1)/n_messages))
#     data = devices_df[(devices_df['experiment_id'] == experiment_id) &
#                              (devices_df['mess_id'] == m_id)][['typ', 't']]
#     if not data.empty:
#         chunk = data[data['type'] == 1]['t']
#         ack = data[data['type'] == 2]['t']
        
#         rtt = None
#         if chunk is not None and ack is not None:
#             rtt = chunk + ack
        
#         transfer_times = transfer_times.append(pd.DataFrame([
#             [chunk, 1],
#             [ack, 2],
#             [rtt, 3]
#         ], columns=[['t', 'type']]), ignore_index=True)
#     else:
#         print("No ({}, {}) in devices_df".format(experiment_id, m_id))
        
def to_cdf(data):
    data_size = len(data)
    data_set=sorted(set(data))
    bins=np.append(data_set, data_set[-1]+1)
    counts, bin_edges = np.histogram(data, bins=bins, density=False)

    counts=counts.astype(float)/data_size

    # Find the cdf
    cdf = np.cumsum(counts)
    
    return cdf, bins

In [ ]:
PLOT_DIR = "outputs/users_number/"
os.makedirs(PLOT_DIR)

In [ ]:
f, ax = plt.subplots()

# Plot the cdf
cdf, bins = to_cdf(transfer_times[transfer_times['type'] == 1]['t'])
ax.plot(bins[0:-1], cdf, label="Chunks")
cdf, bins = to_cdf(transfer_times[transfer_times['type'] == 2]['t'])
ax.plot(bins[0:-1], cdf, label="Acks")
cdf, bins = to_cdf(transfer_times[transfer_times['type'] == 3]['t'])
ax.plot(bins[0:-1], cdf, label="RTT")

ax.set_xlabel('Time (s)')
ax.set_xlim(0,)
ax.set_ylabel('CDF (%)')
ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:.0%}'.format(y))) 
ax.set_ylim(0,)
ax.legend()
plt.suptitle('Cumulative distribution function of messages transmission time')


sns.despine()
plt.savefig('outputs/users_number/messages_rtt_cdf.svg')
plt.show()